In [3]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np
from DataProcess import *
from model import JRNN
from torch.optim import SGD,AdamW
from criterion import JRNNLoss
from runner import Runner
# F 范数实现？


In [5]:
batch_size = 50
max_len = 30
min_len = 3
num_examples = -1
root = './ace05E/'
mode = 'train'
entity_type_dim = 20
hidden_size = 100
num_layers = 2
trg_size = 35
arg_size = 24
num_epoch = 20
weight_trg = torch.tensor([0.3]*2 + [1]*33)
weight_arg = torch.tensor([0.2]*2 + [1]*22)
device = 'cuda:0'

In [6]:
word_vocab,entity_vocab,trg_vocab,arg_vocab,embedding_weight = build_vocab(device = device)
train_loader = load_data(word_vocab,entity_vocab,trg_vocab,arg_vocab,device,batch_size = batch_size, max_len = max_len ,min_len = min_len,num_examples = num_examples,root = root,mode = mode)

In [ ]:
model = JRNN(embedding_weight, entity_type_dim, 
             hidden_size, num_layers, trg_size, arg_size, length = max_len,device=device, dropout = 0.1, bidirectional = True)
model = model.double()
model.to(device)
optimizer = AdamW(model.parameters(),lr= 5e-5)
loss_fn = JRNNLoss(weight_trg=None,weight_arg=None)
loss_fn.to(device)
runner = Runner(model,optimizer,loss_fn) 
runner.train(train_loader, num_epoch)


In [8]:
# ACE05Dataset
# 每行为一个字典，包含key如下：
# doc_id,sent_id,entity_mentions,relation_mentions,event_mentions,tokens,sentence
# 本任务所需内容:
# tokens: 列表: 已分词，区分大小写，包括了标点
# entity_mentions: [ {'id','start','end','entity_type','mention_type','text'} ]
# 经检查: entity_type 包括 {'FAC', 'GPE', 'VEH', 'LOC', 'PER', 'WEA', 'ORG'}, mention_type 只有 'UNK'

# event_mentions: [ {'event_type', 'id', 'trigger': {'start', 'end', 'text'}, 'arguments': [{'entity_id', 'text', 'role'}]} ]
# 这里的trigger长度都是1，意味着每个event_type都只会对应一个长度为1的触发词
# 8-33 subtype: {'Conflict:Attack', 'Justice:Arrest-Jail', 'Transaction:Transfer-Money', 'Life:Marry', 'Life:Injure', 'Business:Merge-Org', 'Business:Start-Org', 'Life:Die', 'Life:Divorce', 'Justice:Release-Parole', 'Personnel:End-Position', 'Justice:Sentence', 'Justice:Pardon', 'Justice:Convict', 'Life:Be-Born', 'Conflict:Demonstrate', 'Contact:Meet', 'Justice:Sue', 'Justice:Charge-Indict', 'Personnel:Start-Position', 'Contact:Phone-Write', 'Justice:Fine', 'Personnel:Nominate', 'Justice:Trial-Hearing', 'Justice:Appeal', 'Justice:Extradite', 'Business:End-Org', 'Justice:Acquit', 'Personnel:Elect', 'Movement:Transport', 'Business:Declare-Bankruptcy', 'Justice:Execute', 'Transaction:Transfer-Ownership'}
# 注意: event_mentions 中可能有多个event_type，对应多个事件 应该作为 label处理
# role: 'Person', 'Prosecutor', 'Recipient', 'Attacker', 'Artifact', 'Origin', 'Entity', 'Vehicle', 'Place', 'Seller', 'Agent', 'Defendant', 'Buyer', 'Giver', 'Beneficiary', 'Victim', 'Adjudicator', 'Target', 'Instrument', 'Org', 'Destination', 'Plaintiff'
class ACE05EDataset(Dataset):
    def __init__(self, root = './ace05E/', mode = 'train') -> None:
        super().__init__()
        data = pd.read_json(root + mode + '.oneie.json', lines=True)
        self.data = data.to_dict(orient='records')
        
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)


In [9]:
trainset = ACE05EDataset()
# print(len(trainset))
# entity_type = set()
# mention_type = set()
# for i in range(len(trainset)):
#     entity_mention = trainset[i]['entity_mentions']
#     for item in entity_mention:
#         entity_type.add(item['entity_type'])
#         mention_type.add(item['mention_type'])
# print(entity_type)
# print(mention_type)

# for idx in range(len(trainset)):
#     print('-------------------')
#     event_mention = trainset[idx]['event_mentions']
#     entity_type = trainset[idx]['entity_mentions']
#     for item in entity_type:
#         print(item['id'],item['text'],end='\t')
#     print()
#     for event in event_mention:
#         print(event['arguments'])
# argset = set()
# for i in range(len(trainset)):
#     event_mention = trainset[i]['event_mentions']
#     for item in event_mention:
#         arguments = item['arguments']
#         for arg in arguments:
#             argset.add(arg['role'])
# print(argset)
# event_type = set()
# for i in range(len(trainset)):
#     event_mention = trainset[i]['event_mentions']
#     for item in event_mention:
#         event_type.add(item['event_type'])
# print(event_type)
# print(len(event_type))

for i in range(20):
    print(trainset[i]['tokens'])

['CNN_CF_20030303.1900.00']
['STORY']
['2003', '-', '03', '-', '03T19:00:00', '-', '05:00']
['New', 'Questions', 'About', 'Attacking', 'Iraq', ';', 'Is', 'Torturing', 'Terrorists', 'Necessary', '?']
['BEGALA', 'Well', ',', 'we', "'ll", 'debate', 'that', 'later', 'on', 'in', 'the', 'show', '.']
['We', "'ll", 'have', 'a', 'couple', 'of', 'experts', 'come', 'out', ',', 'so', 'I', "'ll", 'withhold', 'my', 'comments', 'until', 'then', '.']
['Even', 'as', 'the', 'secretary', 'of', 'homeland', 'security', 'was', 'putting', 'his', 'people', 'on', 'high', 'alert', 'last', 'month', ',', 'a', '30-foot', 'Cuban', 'patrol', 'boat', 'with', 'four', 'heavily', 'armed', 'men', 'landed', 'on', 'American', 'shores', ',', 'utterly', 'undetected', 'by', 'the', 'Coast', 'Guard', 'Secretary', 'Ridge', 'now', 'leads', '.']
['Now', ',', 'why', 'has', 'our', 'president', 'placed', 'homeland', 'security', 'in', 'the', 'hands', 'of', 'Republican', 'political', 'hacks', 'instead', 'of', 'professionals', ',', 'by'